In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import time 
import pandas as pd
import json
import re
from pymongo import MongoClient

In [2]:
hh_main_link = 'https://hh.ru/'
sj_main_link = 'https://russia.superjob.ru'
header = {'Accept': '*/*',
          'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0'}
proxi = {'http': ''}
search_str = 'python'
first_page = 0

In [3]:
def hh(hh_main_link, search_str, first_page):
    data = list()
    html = requests.get(f'{hh_main_link}search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text={search_str}&page={first_page}', headers=header, proxies=proxi).text
    parsed_html = bs(html, 'lxml')
    #Находим номер последней страницы 
    last_page = int(parsed_html.find_all(attrs={'class':'bloko-button HH-Pager-Control'})[-1]['data-page'])

    
    for i in range(last_page+1): 
        vacancy_block = parsed_html.find('div',{'class':'vacancy-serp'} )
        vacancy_list = vacancy_block.findChildren(recursive=False)
        for vacancy in vacancy_list:
            vacancy_dict={}
            a = vacancy.find('a',{'class':'bloko-link HH-LinkModifier'})
            if a != None:
                vacancy_dict['site'] = 'HH'
                vacancy_dict['name'] = a.getText()
                vacancy_dict['link'] = a['href']
                compensation = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).text.replace('\xa0', ' ')
                if not compensation:
                    vacancy_dict['compensation_min'] = 0
                    vacancy_dict['compensation_max'] = 0
                else:
                    compensation = compensation.replace('\xa0', ' ').split('-')
                    compensation_min=compensation[0].replace('руб.','').replace('от','').replace('-','').replace('до','').replace('USD','').replace('EUR','').replace(' ', '').replace('₽', '')
                    vacancy_dict['compensation_min'] =int(compensation_min)
                    if len(compensation)>1:
                        compensation_max=compensation[1].replace('руб.','').replace('от','').replace('-','').replace('до','').replace('USD','').replace('EUR','').replace(' ', '').replace('₽', '')
                        vacancy_dict['compensation_max'] = int(compensation_max)
                    else:
                        compensation_max= 0
                        vacancy_dict['compensation_max'] = compensation_max  
                data.append(vacancy_dict)
        
        first_page +=1
        time.sleep(1)
        html = requests.get(f'{hh_main_link}search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text={search_str}&page={first_page}', headers=header, proxies=proxi).text
        parsed_html = bs(html, 'lxml')
    return data

In [4]:
def sj(sj_main_link, search_str):
    data_sj=list()
    html = requests.get(f'{sj_main_link}/vacancy/search/?keywords={search_str}&geo[c][0]=1', headers=header, proxies=proxi).text
    parsed_sj = bs(html, 'lxml')
    while parsed_sj:
        vacancy_block = parsed_sj.find('div', {'style':"display:block"})
        vacancy_list = vacancy_block.findChildren('div', {'class':'_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'}, recursive=False)
                
        for vacancy in vacancy_list:
            vacancy_dict={}
            b = vacancy.find('a', {'class':'_1QIBo'}) 
            vacancy_dict['site'] = 'SJ'
            vacancy_dict['name'] = b.text
            vacancy_dict['link'] = sj_main_link+b['href']
           
            compensation_block = vacancy.find('span', {'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})
            compensation_sj = compensation_block.text.replace('\xa0', ' ')
            if compensation_sj == 'По договорённости':
                    vacancy_dict['compensation_min'] = 0
                    vacancy_dict['compensation_max'] = 0
            else:
                compensation_sj = compensation_sj.replace(' ','').replace('&nbsp','').replace('₽', '').split('—')
                compensation_min=compensation_sj[0].replace('руб.','').replace('от','').replace('-','').replace('до','').replace('USD','').replace('EUR','').replace(' ', '')
                vacancy_dict['compensation_min'] =int(compensation_min)
                if len(compensation_sj)>1:
                    compensation_max=compensation_sj[1]
                    vacancy_dict['compensation_max'] = int(compensation_max)
                else:
                    compensation_max= 0
                    vacancy_dict['compensation_max'] = compensation_max
            data_sj.append(vacancy_dict)
        next_page_search = parsed_sj.find("a", {'class':"icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe"})
        if next_page_search:
            html=requests.get(sj_main_link+next_page_search['href'], headers=header, proxies=proxi).text
            parsed_sj = bs(html, 'lxml')
                
        else:
            parsed_sj = None
    return (data_sj)

In [5]:
result_sj = sj(sj_main_link, search_str)
len(result_sj)


60

In [6]:
result_hh = hh(hh_main_link, search_str,first_page)
len(result_hh)

2000

In [7]:
total_jobs = result_hh + result_sj

## MongoDB

#### Пришлось долго просидеть с обработкой полей compensation_min и compensation_max,т к если их обрабатывать после добавления в итоговый список, то потом получалось, что при отбрасывании лишних(не цифровых значений) и конвертации их в int получались не понятно откуда взявшиеся для меня ошибки наподобие invalid literal for int() with base 10: '' и т д . Хотя визуально в списке не было ни пустых строк, ни каких либо значений, мешающих переводу в int. Поратил пол дня =\ в итоге за 15 мин убрал эту обработку в изначальный парсер. Правда, на всякий случай убрал и зарплату "по договоренности", что бы привести столбец в базе к одному типу данных, на всякий случай, несмотря на то что целевой запрос db.collection.find({'compensation_min':{'$gt': 100000}}): - отрабатывал и с разными типами данных в этом столбце =)

In [8]:
client = MongoClient('localhost', 27017)
db = client['jobs']
collection = db.hhsj
db.collection.insert_many(total_jobs)

In [9]:
#db.collection.deleteMany({})

In [10]:
for i in db.collection.find({'compensation_min':{'$gt': 100000}}):
    pprint(i)

{'_id': ObjectId('5e77d591b4036c10102db623'),
 'compensation_max': 200000,
 'compensation_min': 150000,
 'link': 'https://spb.hh.ru/vacancy/36396758?query=python',
 'name': 'Team Lead - Python',
 'site': 'HH'}
{'_id': ObjectId('5e77d591b4036c10102db625'),
 'compensation_max': 200000,
 'compensation_min': 150000,
 'link': 'https://spb.hh.ru/vacancy/36202473?query=python',
 'name': 'Fullstack-разработчик (Django (python) + React)',
 'site': 'HH'}
{'_id': ObjectId('5e77d591b4036c10102db626'),
 'compensation_max': 0,
 'compensation_min': 250000,
 'link': 'https://spb.hh.ru/vacancy/36317669?query=python',
 'name': 'C# техлид (back-end, teamlead)',
 'site': 'HH'}
{'_id': ObjectId('5e77d591b4036c10102db62a'),
 'compensation_max': 200000,
 'compensation_min': 160000,
 'link': 'https://spb.hh.ru/vacancy/35569509?query=python',
 'name': 'Программист Python',
 'site': 'HH'}
{'_id': ObjectId('5e77d591b4036c10102db62d'),
 'compensation_max': 0,
 'compensation_min': 120000,
 'link': 'https://spb.hh.

 'link': 'https://spb.hh.ru/vacancy/36344503?query=python',
 'name': 'Project manager',
 'site': 'HH'}
{'_id': ObjectId('5e77d591b4036c10102dbb62'),
 'compensation_max': 0,
 'compensation_min': 250000,
 'link': 'https://spb.hh.ru/vacancy/34072020?query=python',
 'name': 'Программист систем машинного (компьютерного) зрения',
 'site': 'HH'}
{'_id': ObjectId('5e77d591b4036c10102dbb63'),
 'compensation_max': 0,
 'compensation_min': 150000,
 'link': 'https://spb.hh.ru/vacancy/34480650?query=python',
 'name': 'QA automation',
 'site': 'HH'}
{'_id': ObjectId('5e77d591b4036c10102dbb86'),
 'compensation_max': 200000,
 'compensation_min': 160000,
 'link': 'https://spb.hh.ru/vacancy/34561068?query=python',
 'name': 'Старший аналитик BI (Tableau)',
 'site': 'HH'}
{'_id': ObjectId('5e77d591b4036c10102dbb93'),
 'compensation_max': 0,
 'compensation_min': 190000,
 'link': 'https://spb.hh.ru/vacancy/35626883?query=python',
 'name': 'DevOps Developer',
 'site': 'HH'}
{'_id': ObjectId('5e77d591b4036c101

 'link': 'https://spb.hh.ru/vacancy/36360039?query=python',
 'name': 'Golang ведущий разработчик',
 'site': 'HH'}
{'_id': ObjectId('5e77d8d825ad52800d217624'),
 'compensation_max': 0,
 'compensation_min': 130000,
 'link': 'https://spb.hh.ru/vacancy/36134932?query=python',
 'name': 'Разработчик Python',
 'site': 'HH'}
{'_id': ObjectId('5e77d8d825ad52800d217629'),
 'compensation_max': 180000,
 'compensation_min': 180000,
 'link': 'https://spb.hh.ru/vacancy/36202989?query=python',
 'name': 'Back-end Developer Python',
 'site': 'HH'}
{'_id': ObjectId('5e77d8d825ad52800d21762f'),
 'compensation_max': 0,
 'compensation_min': 180000,
 'link': 'https://spb.hh.ru/vacancy/36213343?query=python',
 'name': 'QA Engineer Python',
 'site': 'HH'}
{'_id': ObjectId('5e77d8d825ad52800d217637'),
 'compensation_max': 150000,
 'compensation_min': 120000,
 'link': 'https://spb.hh.ru/vacancy/35133105?query=python',
 'name': 'Разработчик (Python)',
 'site': 'HH'}
{'_id': ObjectId('5e77d8d825ad52800d21763d'),
 

 'site': 'HH'}
{'_id': ObjectId('5e77d8d825ad52800d217c96'),
 'compensation_max': 130000,
 'compensation_min': 130000,
 'link': 'https://spb.hh.ru/vacancy/36254533?query=python',
 'name': 'Сетевой инженер & Linux администратор',
 'site': 'HH'}
{'_id': ObjectId('5e77d8d825ad52800d217c97'),
 'compensation_max': 180000,
 'compensation_min': 150000,
 'link': 'https://spb.hh.ru/vacancy/36229254?query=python',
 'name': 'Backend Developer (senior/lead) / Python',
 'site': 'HH'}
{'_id': ObjectId('5e77d8d825ad52800d217ca2'),
 'compensation_max': 0,
 'compensation_min': 160000,
 'link': 'https://spb.hh.ru/vacancy/36215592?query=python',
 'name': 'Devops специалист',
 'site': 'HH'}
{'_id': ObjectId('5e77d8d825ad52800d217cb8'),
 'compensation_max': 0,
 'compensation_min': 150000,
 'link': 'https://spb.hh.ru/vacancy/36194063?query=python',
 'name': 'Ведущий Web-разработчик PHP/JS (full stack)',
 'site': 'HH'}
{'_id': ObjectId('5e77d8d825ad52800d217cc1'),
 'compensation_max': 0,
 'compensation_min':